In [1]:
import pandas as pd

df = pd.read_csv("hist_routing_orders.csv")
df.head()

,tick,route_name,type,order_lat,order_lng,finish_tick,open_time,close_time,is_delivery,order_status,carrier_id,carrier_lat,carrier_lng,order_id
0,286,136,PLANNED,32.69714,-117.07711,16,0,1439,True,Complete,0,32.72329,-117.07189,planned_0161
1,286,136,PLANNED,32.68411,-117.04626,35,0,1439,True,Complete,0,32.72329,-117.07189,planned_0162
2,286,136,PLANNED,32.67110,-117.09170,59,0,1439,True,Complete,0,32.72329,-117.07189,planned_0163
3,286,136,PLANNED,32.72329,-117.07189,88,0,1439,NaN,Dummy,0,32.72329,-117.07189,dummy_rtb_order
4,286,137,PLANNED,32.73967,-117.19347,31,0,1439,True,Complete,1,32.72329,-117.07189,planned_0164


In [7]:
import random
import numpy as np
from datetime import datetime, timedelta
start_date = datetime(2022,2,1,7,59)
colors = ['blue', 'black', 'darkblue', 'purple']
oncall_order_color = 'green'
terminated_order_color = "red"
delayed_order_color = 'orange'
last_tick = 1439

station_loc = [32.72329226, -117.0718922]

def get_stats(tick):
    df_stat = df[((df['tick'] == last_tick) & (df['finish_tick'] <= tick))]
    df_stat = df_stat[df_stat['tick'] == df_stat['tick'].max()]
    order_status_stat = df_stat[['order_id', 'order_status']].groupby('order_status').count().reset_index()
    order_status_stat = {row['order_status']: row['order_id'] for _, row in order_status_stat.iterrows()}
    info = ""
    for key, val in order_status_stat.items():
        info += f"{key}: {val}, "
    return info

def create_geojson_features(_route_name=None):
    if _route_name:
        route_name_list = [_route_name]
    else:
        route_name_list = np.random.choice(df[df['type'] == "ONCALL"]['route_name'].unique(), size=3)
    print('> Creating GeoJSON features...')
    features = []
    for route_name in route_name_list:
        df_route = df[df['route_name'] == route_name]
        df_route_tick = df_route[df_route['tick'] == last_tick]
        df_route_tick.sort_values(by='finish_tick', inplace=True)
        points = [[station_loc[1], station_loc[0]]]
        ticks_to_visit = [(start_date+timedelta(minutes=1)).__str__()]
        route_color = colors[int(route_name)%len(colors)]
        for _, row in df_route_tick.iterrows():
            if row['type'] != 'PLANNED':
                if row['order_status'] == "Delayed":
                    color = delayed_order_color
                elif row['order_status'] == 'Terminated':
                    color = terminated_order_color
                else:
                    color = oncall_order_color
            else:
                color = route_color
            if row['order_status'] == 'Complete':
                points.append([row['order_lng'], row['order_lat']])
                ticks_to_visit.append((start_date+timedelta(minutes=row['finish_tick'])).__str__())
            delivery = ("deliver" if row['is_delivery'] else "pickup")
            popup = f"{delivery}, {row['order_status']}"
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type':'Point', 
                    'coordinates':[row['order_lng'],row['order_lat']]
                },
                'properties': {
                    'time': (start_date+timedelta(minutes=row['finish_tick'])).__str__(),
                    'style': {'color' : color},
                    'icon': 'circle',
                    "popup": popup,
                    'iconstyle':{
                        'fillColor': color,
                        'fillOpacity': 0.8,
                        'stroke': 'false',
                        'radius': 5
                    }
                }
            }
            features.append(feature)
        for i, finished_tick in enumerate(df_route_tick['finish_tick'].unique()):
            finished_tick = int(finished_tick)
            stat_info = get_stats(finished_tick)
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type':'Point', 
                    'coordinates':[station_loc[1], station_loc[0]]
                },
                'properties': {
                    'time': (start_date+timedelta(minutes=finished_tick)).__str__(),
                    'style': {'color' : "green"},
                    'icon': 'circle',
                    "popup": stat_info,
                    'iconstyle':{
                        'fillColor': "green",
                        'fillOpacity': (1.0 if i == 0 else 0.0),
                        'stroke': 'false',
                        'radius': 10
                    }
                }
            }
            features.append(feature)
        features.append(
            {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": points,
                },
                "properties": {
                    "times": ticks_to_visit,
                    "style": {
                        "color": route_color,
                        "weight": 5,
                    },
                    'iconstyle':{
                        'fillColor': route_color,
                        'fillOpacity': 0.8,
                        'stroke': 'true',
                        'radius': 7
                }
                },
            }
        )
    return features

import folium
from folium.plugins import TimestampedGeoJson

def make_map(features):
    print('> Making map...')
    routing_map = folium.Map(location=station_loc, control_scale=True, zoom_start=12)
    # popup = "I am a station!"
    # folium.Marker(location=station_loc, 
    #                        popup=popup,
    #                        icon=folium.Icon(color="green", icon="star")).add_to(routing_map)
    TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period='PT1M'
        , add_last_point=True
        , auto_play=False
        , loop=False
        , max_speed=1
        , loop_button=True
        , date_options='YYYY-MM-DD HH:mm'
        , time_slider_drag_update=True
    ).add_to(routing_map)
    print('> Done.')
    return routing_map

features = create_geojson_features()
routing_map = make_map(features)
routing_map.save("routing.html")

> Creating GeoJSON features...


/opt/anaconda3/envs/maroenv/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


> Making map...
> Done.
